In [20]:
NEO4J_URI = "neo4j+s://7d3d4dbd.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "4cQimVVQmSVbF5rh6BA-qjWJcIQkmRnyjKXMoxco0Pw"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def test_connection():
    try:
        with driver.session() as session:
            result = session.run("RETURN 1")
            return result.single()[0]
    except Exception as e:
        return f"Erreur de connexion : {str(e)}"
    finally:
        driver.close()

print(test_connection())


1
